# Chapter 10: Introduction to Artificial Neural Networks with Keras

## 1. Chapter Overview
**Tujuan:** Bab ini menandai transisi dari Machine Learning tradisional ke **Deep Learning**. Kita akan mempelajari Jaringan Saraf Tiruan (Artificial Neural Networks - ANN), mulai dari arsitektur paling sederhana (Perceptron) hingga Multi-Layer Perceptron (MLP). Kita akan menggunakan **Keras**, API tingkat tinggi yang berjalan di atas TensorFlow, untuk membangun model yang dapat mengklasifikasikan gambar fashion dan memprediksi harga rumah.

**Konsep Kunci:**
* **Perceptron & TLU:** Unit dasar logika jaringan saraf.
* **Multi-Layer Perceptron (MLP):** Menumpuk lapisan untuk memecahkan masalah non-linear (XOR problem).
* **Backpropagation:** Algoritma pelatihan revolusioner yang menghitung gradien secara efisien.
* **Fungsi Aktivasi:** Mengapa kita membutuhkan ReLU, Sigmoid, atau Softmax.
* **Keras API:**
    * *Sequential API:* Untuk tumpukan lapisan sederhana.
    * *Functional API:* Untuk topologi kompleks (Wide & Deep).
    * *Subclassing API:* Untuk kontrol penuh (research level).
* **Hyperparameter Tuning:** Menggunakan RandomizedSearch untuk mencari jumlah neuron dan learning rate yang optimal.

**Keterampilan Praktis:**
* Membangun Image Classifier untuk dataset **Fashion MNIST**.
* Membangun Regressor untuk dataset perumahan California.
* Menggunakan Callbacks (EarlyStopping, ModelCheckpoint) untuk mencegah overfitting.
* Menyimpan dan memuat model Keras.

In [ ]:
# Setup
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

import numpy as np
import os
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

## 2. Penjelasan Teoretis (Mendalam)

### 1. Dari Biologis ke Buatan (Perceptron)
Ide dasar ANN terinspirasi dari neuron biologis. Neuron menerima sinyal input melalui dendrit, memprosesnya, dan jika sinyal cukup kuat, menembakkan sinyal output melalui akson.

**Perceptron (Frank Rosenblatt, 1957):**
Ini adalah arsitektur ANN paling sederhana. Terdiri dari satu lapisan **Threshold Logic Units (TLU)**. Input adalah angka, masing-masing memiliki bobot ($w$). TLU menghitung jumlah bobot ($z = w_1 x_1 + \dots + w_n x_n + bias$), lalu menerapkan *step function*.
* Kelemahan: Perceptron hanya bisa memecahkan masalah linear. Ia gagal pada masalah XOR sederhana.

### 2. Multi-Layer Perceptron (MLP) dan Backpropagation
Untuk mengatasi batasan Perceptron, kita menumpuk banyak lapisan TLU. Struktur ini disebut MLP:
1.  **Input Layer:** Menerima fitur data.
2.  **Hidden Layers:** Satu atau lebih lapisan di tengah yang melakukan transformasi representasi.
3.  **Output Layer:** Menghasilkan prediksi akhir.

**Backpropagation (Rumelhart, Hinton, Williams, 1986):**
Bagaimana cara melatih jaringan sedalam ini? Backpropagation adalah kuncinya. Algoritma ini bekerja dalam dua langkah utama untuk setiap *batch* pelatihan:
1.  **Forward Pass:** Data mengalir dari input ke output, menghasilkan prediksi, dan kesalahan (error) dihitung menggunakan *Loss Function*.
2.  **Backward Pass:** Algoritma menghitung gradien kesalahan terhadap setiap parameter jaringan (bobot) dengan bergerak mundur dari output ke input (menggunakan *Chain Rule* kalkulus). Ini memberi tahu kita seberapa besar setiap bobot berkontribusi terhadap kesalahan.
3.  **Update:** Langkah Gradient Descent menggunakan gradien tersebut untuk memperbarui bobot agar kesalahan berkurang.

### 3. Fungsi Aktivasi
Agar MLP bisa mempelajari pola non-linear, kita **harus** menggunakan fungsi aktivasi non-linear di antara lapisan linear.
* **Sigmoid:** Mengubah output menjadi rentang 0 hingga 1. Bagus untuk output probabilitas, tapi menderita masalah *vanishing gradient* pada lapisan dalam.
* **ReLU (Rectified Linear Unit):** $ReLU(z) = max(0, z)$. Sangat cepat dihitung dan tidak jenuh untuk nilai positif. Ini adalah standar *de facto* untuk hidden layers saat ini.
* **Softmax:** Digunakan di *output layer* untuk klasifikasi multikelas. Memastikan total probabilitas semua kelas berjumlah 1.

### 4. Keras API
TensorFlow 2 menjadikan Keras sebagai API resminya. Keras sangat *user-friendly*:
* **Sequential API:** Paling mudah. Cukup `model.add(Layer)`. Cocok untuk 90% kasus.
* **Functional API:** Lebih fleksibel. Memungkinkan input ganda, output ganda, dan topologi non-linear (seperti arsitektur Wide & Deep).
* **Subclassing API:** Paling kompleks tapi paling fleksibel. Anda menulis kelas Python sendiri. Digunakan untuk riset model baru.

## 3. Reproduksi Kode

### 3.1 Membangun Image Classifier (Fashion MNIST)
Kita akan menggunakan dataset Fashion MNIST (70.000 gambar grayscale 28x28 dari 10 kategori pakaian) karena sedikit lebih sulit daripada digit MNIST biasa.

In [ ]:
# 1. Load Data
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

# 2. Validasi Split & Normalisasi (Scaling)
# Pixel values are 0-255. We scale them to 0-1 for Neural Networks.
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.0

class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

print("Shape data train:", X_train.shape)
print("Contoh kelas:", class_names[y_train[0]])

# Visualisasi satu gambar
plt.imshow(X_train[0], cmap="binary")
plt.axis('off')
plt.show()

### 3.2 Membuat Model dengan Sequential API
Kita akan membuat MLP dengan 2 hidden layer.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]), # Mengubah 2D (28x28) menjadi 1D (784)
    keras.layers.Dense(300, activation="relu"), # Hidden Layer 1: 300 neuron, ReLU
    keras.layers.Dense(100, activation="relu"), # Hidden Layer 2: 100 neuron, ReLU
    keras.layers.Dense(10, activation="softmax") # Output Layer: 10 kelas, Softmax
])

# Melihat ringkasan arsitektur
model.summary()

### 3.3 Compile dan Train Model
* **Loss:** `sparse_categorical_crossentropy` karena label kita berupa integer (0-9), bukan one-hot vector.
* **Optimizer:** `sgd` (Stochastic Gradient Descent).
* **Metrics:** `accuracy`.

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

# Melatih model (ini mungkin memakan waktu beberapa detik/menit)
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid))

### 3.4 Visualisasi Kurva Pembelajaran
Objek `history` menyimpan data loss dan akurasi selama pelatihan. Kita bisa memplotnya untuk melihat apakah terjadi overfitting.

In [ ]:
import pandas as pd

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # Set range sumbu Y dari 0 ke 1
plt.title("Learning Curves")
plt.show()

# Evaluasi pada Test Set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {test_acc:.4f}")

### 3.5 Prediksi
Menggunakan model untuk memprediksi probabilitas kelas pada data baru.

In [ ]:
X_new = X_test[:3]
y_proba = model.predict(X_new)
print("Probabilitas:\n", y_proba.round(2))

# Mengambil kelas dengan probabilitas tertinggi
y_pred = np.argmax(y_proba, axis=1)
print("Prediksi:", np.array(class_names)[y_pred])
print("Label Asli:", np.array(class_names)[y_test[:3]])

### 3.6 Regression MLP (Functional API)
Untuk regresi (memprediksi harga rumah California), struktur MLP sedikit berbeda:
* Output layer hanya punya **1 neuron**.
* **Tidak ada fungsi aktivasi** di output (karena kita ingin nilai kontinu bebas).
* Loss function: **MSE**.

Kita juga akan menggunakan **Functional API** untuk membuat arsitektur **Wide & Deep**. Arsitektur ini menghubungkan sebagian input langsung ke output (Wide) dan sebagian melalui hidden layers (Deep). Ini memungkinkan model mempelajari pola sederhana (linear) dan pola kompleks (deep) sekaligus.

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load & Split Data
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

# Scaling (Sangat PENTING untuk Neural Networks!)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# --- Functional API ---
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)

# Concatenate input langsung dengan output hidden layer 2
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)

model_func = keras.models.Model(inputs=[input_], outputs=[output])

model_func.compile(loss="mean_squared_error", optimizer="sgd")

print("Training Wide & Deep Model...")
history = model_func.fit(X_train, y_train, epochs=20,
                         validation_data=(X_valid, y_valid))

### 3.7 Callbacks
Apa yang terjadi jika kita melatih terlalu lama? Overfitting. Daripada menebak jumlah epoch, kita gunakan `EarlyStopping`. Kita juga gunakan `ModelCheckpoint` untuk menyimpan model terbaik.

In [ ]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)

# Patience=10: Hentikan jika tidak ada perbaikan pada val_loss selama 10 epoch berturut-turut
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

model_func.fit(X_train, y_train, epochs=100,
               validation_data=(X_valid, y_valid),
               callbacks=[checkpoint_cb, early_stopping_cb])

## 4. Langkah-demi-Langkah Penjelasan

### 1. Preprocessing Data Gambar
**Input:** Gambar `28x28` pixel dengan nilai `0-255`.
**Proses:** Neural Network bekerja paling baik dengan input angka kecil (sekitar 0 hingga 1). Oleh karena itu, kita membagi data dengan `255.0`. Jika kita lupa langkah ini, model mungkin akan gagal konvergen (gradien meledak atau menghilang).

### 2. Arsitektur Model Klasifikasi
* `Flatten`: Lapisan ini tidak memiliki parameter (bobot) untuk dipelajari. Tugasnya hanya "meratakan" matriks 2D menjadi vektor panjang. Ini adalah jembatan antara data gambar dan lapisan Dense.
* `Dense`: Setiap neuron di layer ini terhubung ke *semua* neuron di layer sebelumnya. Inilah mengapa disebut *Dense* (padat) atau *Fully Connected*.
* `ReLU`: Tanpa ini, tumpukan lapisan Dense hanyalah satu fungsi linear besar ($f(g(x)) = W_2(W_1 x)$ masih linear). ReLU memperkenalkan non-linearitas, memungkinkan model mempelajari batas keputusan yang kompleks (seperti lengkungan).
* `Softmax`: Mengubah skor mentah (logits) dari output layer menjadi probabilitas yang valid (total = 1).

### 3. Arsitektur Wide & Deep
Pada Functional API, kita melihat `Concatenate`. Idenya adalah:
* Jalur **Deep** (lewat hidden layer) belajar pola kompleks dan abstrak.
* Jalur **Wide** (langsung dari input ke output) belajar pola sederhana atau aturan eksplisit.
* Menggabungkan keduanya seringkali menghasilkan performa yang lebih baik daripada MLP standar untuk data tabular.

### 4. Early Stopping
Ini adalah bentuk regularisasi otomatis. Alih-alih Anda harus memonitor grafik loss secara manual dan menekan tombol stop, callback ini melakukannya untuk Anda. Jika `val_loss` tidak turun selama 10 epoch, ia berasumsi model sudah mulai overfitting (atau macet), menghentikan pelatihan, dan mengembalikan bobot ke kondisi terbaik sebelumnya.

## 5. Ringkasan Bab

* **Deep Learning** menggunakan jaringan saraf berlapis banyak untuk mempelajari representasi data yang kompleks.
* **Keras** adalah API yang sangat kuat dan mudah digunakan untuk membangun model deep learning di TensorFlow.
* **Sequential API** sangat bagus untuk model tumpukan sederhana.
* **Functional API** diperlukan untuk arsitektur yang lebih kompleks (cabang, multiple input/output).
* **Preprocessing:** Selalu lakukan penskalaan (StandardScaler/MinMax) pada data sebelum dimasukkan ke Neural Network.
* **Loss Function:** Gunakan `sparse_categorical_crossentropy` untuk klasifikasi integer label, dan `mean_squared_error` untuk regresi.
* **Activation:** Gunakan `ReLU` untuk hidden layers dan `Softmax` (klasifikasi) atau `Linear/None` (regresi) untuk output layer.
* **Optimization:** Jangan biarkan model overfitting; gunakan `EarlyStopping` dan simpan model terbaik dengan `ModelCheckpoint`.